In [ ]:
%pip install --upgrade 'deltabase' 'deltabase[magic]' 'deltabase[ai]' requests

In [2]:
from deltabase import delta

from polars import DataFrame
from requests import get

In [3]:
db:delta = delta.connect("delta")
db.config.dtype = "polars"

In [ ]:
bulk_data = get("https://api.scryfall.com/bulk-data").json()
download_uri = next((item['download_uri'] for item in bulk_data['data'] if item['type'] == 'all_cards'), None)

data = get(download_uri).json()
data = DataFrame(data)

In [5]:
err = db.upsert(table="colors", primary_key="id", data=data[[
    "id", "lang", "set_name", "rarity", "colors"
]].explode("colors").fill_null(strategy="zero"))
assert not err, err
err = db.commit(table="colors", partition_by=["lang", "set_name", "rarity"])
assert not err, err

err = db.upsert(table="images", primary_key="id", data=data[[
    "id", "lang", "set_name", "rarity", "image_uris"
]].unnest("image_uris").fill_null(strategy="zero"))
assert not err, err
err = db.commit(table="images", partition_by=["lang", "set_name", "rarity"])
assert not err, err

err = db.upsert(table="keywords", primary_key="id", data=data[[
    "id", "lang", "set_name", "rarity", "keywords"
]].explode("keywords").fill_null(strategy="zero"))
assert not err, err
err = db.commit(table="keywords", partition_by=["lang", "set_name", "rarity"])
assert not err, err

err = db.upsert(table="legalities", primary_key="id", data=data[[
    "id", "lang", "set_name", "rarity", "legalities"
]].unnest("legalities").fill_null(strategy="zero"))
assert not err, err
err = db.commit(table="legalities", partition_by=["lang", "set_name", "rarity"])
assert not err, err

err = db.upsert(table="prices", primary_key="id", data=data[[
    "id", "lang", "set_name", "rarity", "prices"
]].unnest("prices").drop("usd_etched").fill_null(strategy="zero"))
assert not err, err
err = db.commit(table="prices", partition_by=["lang", "set_name", "rarity"])
assert not err, err

err = db.upsert(table="produced_mana", primary_key="id", data=data[[
    "id", "lang", "set_name", "rarity", "produced_mana"
]].explode("produced_mana").fill_null("zero"))
assert not err, err
err = db.commit(table="produced_mana", partition_by=["lang", "set_name", "rarity"])
assert not err, err

In [6]:
data = data.drop([
    "all_parts",
    "arena_id",
    "artist_ids",
    "card_faces",
    "cardmarket_id",
    "color_identity",
    "colors",
    "finishes",
    "frame_effects",
    "games",
    "highres_image",
    "image_status",
    "image_uris",
    "keywords",
    "layout",
    "legalities",
    "mtgo_foil_id",
    "mtgo_id",
    "multiverse_ids",
    "object",
    "oracle_id",
    "oracle_text",
    "preview",
    "prices",
    "printed_name",
    "printed_text",
    "printed_type_line",
    "prints_search_uri",
    "produced_mana",
    "promo_types",
    "purchase_uris",
    "related_uris",
    "rulings_uri",
    "scryfall_set_uri",
    "scryfall_uri",
    "security_stamp",
    "set_search_uri",
    "set_uri",
    "uri",
])

err = db.upsert(table="cards", primary_key="id", data=data.fill_null("zero"))
assert not err, err
err = db.commit(table="cards", partition_by=["lang", "set_name", "rarity"])
assert not err, err

In [7]:
data = db.sql("""
select
    p.id, 
    c.name,
    c.set_name,
    c.released_at as release_date,
    cast(p.usd as float) as usd,
from prices as p
inner join cards as c on c.id = p.id
where p.usd != '' and c.lang = 'en'
order by usd desc
""")
err = db.upsert(table="card_value", primary_key="id", data=data)
assert not err, err
err = db.commit(table="card_value", partition_by=["set_name"])
assert not err, err

In [8]:
data = db.sql("""
select
    set_name,
    min(cast(release_date as date)) as release_date,
    sum(usd) as usd,
    count(id) as n_cards,
from card_value
group by set_name
order by usd desc
""")
err = db.upsert(table="set_value", primary_key="set_name", data=data)
assert not err, err
err = db.commit(table="set_value")
assert not err, err

---

In [9]:
%%ai
summarize the data available to me.

You have access to the following sets of data related to card games:

1. **Card Values (`card_value`)**: Contains information about individual card values, including identifiers, names, set names, release dates, and their USD prices.

2. **Cards (`cards`)**: Provides detailed attributes of each card, including ID, TCGPlayer ID, name, language, release date, mana cost, converted mana cost (cmc), type, rarity, various boolean flags (e.g., reserved, foil, promo), and attributes like power and toughness.

3. **Colors (`colors`)**: Lists information on card colors, including identifiers, language, rarity, and color specifications.

4. **Images (`images`)**: Contains image URLs for cards, which include different sizes (small, normal, large) and specific formats (PNG, art crop, border crop) alongside their associated metadata like rarity.

5. **Keywords (`keywords`)**: Features keywords associated with cards, giving insight into specific abilities or traits based on language and rarity.

6. **Legalities (`legalities`)**: Specifies the legality of cards in various formats (e.g., standard, modern, commander) based on identifiers, language, rarity, and specific formats.

7. **Prices (`prices`)**: Presents pricing information, including USD prices for normal and foil cards, as well as EURO prices and ticket values.

8. **Produced Mana (`produced_mana`)**: Lists details on mana production for cards, including identifiers, languages, set names, rarity, and the type of mana produced.

9. **Set Values (`set_value`)**: Summarizes data for card sets, detailing the set name, release date, total USD value of the set, and the number of cards in that set.

This data collectively allows you to gain insights into individual cards, their values, legalities, and related multimedia, among other aspects of the card game ecosystem.

In [10]:
db.register(table="card_value", version=0, alias="temp_card_value")

In [11]:
db.tables

['card_value',
 'cards',
 'colors',
 'images',
 'keywords',
 'legalities',
 'prices',
 'produced_mana',
 'set_value',
 'temp_card_value']

In [12]:
%%sql
select 
    c.id, 
    c.name, 
    c.set_name,
    t.usd as previous,
    c.usd as current,
    (c.usd - t.usd) as diff
from card_value as c
inner join temp_card_value as t on c.id = t.id
order by diff desc;

id,name,set_name,previous,current,diff
str,str,str,f64,f64,f64
"""093e3fc5-b2e0-4376-b8ad-4470e0…","""Gratuitous Violence""","""Conspiracy: Take the Crown""",2.14,2.14,0.0
"""43ab3ff8-91b0-437c-9c4b-e9c103…","""Omenspeaker""","""Conspiracy: Take the Crown""",0.06,0.06,0.0
"""9de3eeae-22a5-4d9c-afd6-1cc441…","""Coordinated Assault""","""Conspiracy: Take the Crown""",0.08,0.08,0.0
"""f9083583-6fa9-4b8a-86bb-59e51a…","""Exotic Orchard""","""Conspiracy: Take the Crown""",0.33,0.33,0.0
"""160d39b0-76c5-4218-97e5-5903f7…","""Opaline Unicorn""","""Conspiracy: Take the Crown""",0.09,0.09,0.0
…,…,…,…,…,…
"""c2c6e29e-261a-4953-bdbe-cce879…","""Tamiyo, Field Researcher Emble…","""Eldritch Moon Tokens""",0.71,0.71,0.0
"""dbd994fc-f3f0-4c81-86bd-14ca63…","""Human""","""Eldritch Moon Tokens""",0.29,0.29,0.0
"""e44aa879-b63b-497c-9c1b-233395…","""Zombie""","""Eldritch Moon Tokens""",0.25,0.25,0.0


In [13]:
db.register(table="set_value", version=0, alias="temp_set_value")

In [14]:
%%sql
select
    c.set_name,
    t.usd as previous,
    c.usd as current,
    (c.usd - t.usd) as diff
from set_value as c
inner join temp_set_value as t on c.set_name = t.set_name
order by diff desc;

set_name,previous,current,diff
str,f64,f64,f64
"""Unlimited Edition""",26503.5,26503.5,0.0
"""Limited Edition Beta""",22377.69,22377.69,0.0
"""Limited Edition Alpha""",19426.07,19426.07,0.0
"""Arabian Nights""",13772.22,13772.22,0.0
"""Secret Lair Drop""",13739.5,13739.5,0.0
…,…,…,…
"""Duel Decks: Sorin vs. Tibalt T…",0.12,0.12,0.0
"""Streets of New Capenna Tokens""",0.11,0.11,0.0
"""Duel Decks: Jace vs. Chandra T…",0.1,0.1,0.0
